In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from functools import partial
import dgl

Using backend: pytorch


In [83]:
# load graph data
from dgl.contrib.data import load_data
import numpy as np
data = load_data('FB15k-237')
num_nodes = data.num_nodes
num_rels = data.num_rels

# entities: 14541
# relations: 237
# edges: 272115


In [84]:
len(data.test)

20466

In [85]:
import pandas as pd

In [86]:
data=pd.DataFrame(data.test, columns=['source','edge','target'])

In [99]:
edge = data['edge'].value_counts()[:8].index
edge = pd.DataFrame(edge, columns=['edge'])
edge['relation'] = ['INTERESTED_IN','BOOKMARK','TALK_OF','ASSUME_UNDERSTANDING_OF','WRITTEN_BY','ENCAPSULATES','SYNONYM','REPOSITORY_OF']

In [100]:
edge_dict = dict(zip(edge.edge,edge.relation))

In [148]:
edge_dict

{75: 'INTERESTED_IN',
 2: 'BOOKMARK',
 125: 'TALK_OF',
 108: 'ASSUME_UNDERSTANDING_OF',
 82: 'WRITTEN_BY',
 4: 'ENCAPSULATES',
 59: 'SYNONYM',
 111: 'REPOSITORY_OF'}

In [101]:
data['edge'].value_counts()[:8].index

Int64Index([75, 2, 125, 108, 82, 4, 59, 111], dtype='int64')

In [102]:
data['edge_name'] = data.edge.map(edge_dict)

In [106]:
data = data.dropna()

In [132]:
inter_in = data[data.edge==75].reset_index(drop=True)
inter_in['source_name'] = 'user_'+inter_in['source'].astype(str)
inter_in['target_name'] = 'topic_'+inter_in['target'].astype(str)

In [145]:
bookmark = data[data.edge==2].reset_index(drop=True)
bookmark['source_name'] = 'user_'+bookmark['source'].astype(str)
content = bookmark[bookmark.target<5100]
content['target_name'] = 'content_'+content['target'].astype(str)
extRes = bookmark[bookmark.target>=5100]
extRes['target_name'] = 'externalResources_'+extRes['target'].astype(str)
bookmark = extRes.append(content).reset_index(drop=True)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [162]:
talk_of = data[data.edge==125]

In [170]:
content = talk_of[talk_of.source>8000]
content['source_name'] = 'content_'+content['source'].astype(str)
content['target_name'] = 'topic_'+content['target'].astype(str)
thread = talk_of[talk_of.source<=8000]
thread['source_name'] = 'thread_'+thread['source'].astype(str)
thread['target_name'] = 'book_'+thread['target'].astype(str)
talk_of = thread.append(content)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [192]:
auo = data[data.edge==108]
auo['target_name'] = 'topic_'+auo['target'].astype(str)
content = auo[auo.source>7000]
content['source_name'] = 'content_'+content['source'].astype(str)
topic = auo[auo.source<=7000]
topic['source_name'] = 'topic_'+topic['source'].astype(str)
auo = topic.append(content)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [195]:
written_by = data[data.edge==82]
written_by['source_name'] = 'content_'+written_by['source'].astype(str)
written_by['target_name'] = 'user_'+written_by['target'].astype(str)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [227]:
encaps = data[data.edge==4]
encaps['source_name'] = 'library_'+encaps['source'].astype(str)
method = encaps[encaps.target>12000]
method['target_name'] = 'mlStatMethod_'+method.target.astype(str)
algo = encaps[(encaps.target>4500)&(encaps.target<=12000)]
algo['target_name'] = 'mlStatAlgo_'+algo.target.astype(str)
metric = encaps[encaps.target<=4500]
metric['target_name'] = 'mlStatMetric_'+metric.target.astype(str)
encaps=metric.append([algo,method])

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [228]:
syn = data[data.edge==59]
syn['source_name'] = 'topic_'+syn.source.astype(str)
syn['target_name'] = 'topic_'+syn.target.astype(str)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [229]:
rep_of = data[data.edge==59]
rep_of['source_name'] = 'externalResources_'+syn.source.astype(str)
rep_of['target_name'] = 'library_'+rep_of.target.astype(str)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [230]:
data = inter_in.append([bookmark,talk_of,auo,written_by,encaps,syn,rep_of]).reset_index(drop=True)

C:\Aplikasi\Anaconda\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [233]:
data[['source','edge','target','source_name','edge_name','target_name']].to_csv('sample_dataset_learnavi.csv')